In [1]:
import math
import torch
import pandas as pd

from model.gnn_recommender import GNNRecommender

In [2]:
def get_user_blog_history(data, user_id, blog_df, rating_df):
    blogs = []
    for blog_id in data.edge_index[1, data.edge_index[0] == user_id].tolist():
        rating = rating_df.query('blog_id == @blog_id and userId == @user_id')['ratings'].item()
        full_blog = blog_df.query('blog_id == @blog_id')
        title = full_blog['blog_title'].item()
        topic = full_blog['topic'].item()
        blogs.append((rating, topic, title))
    blogs.sort(reverse = True)
    return blogs

In [3]:
def recommend_blogs(model, data, user_id, top_k=5):
    model.eval()
    with torch.no_grad():
        # All blogs
        all_blogs = torch.arange(data.x.size(0))
        
        # Blogs the user has interacted with
        interacted_blogs = data.edge_index[1, data.edge_index[0] == user_id].unique()
        
        # Blogs the user has not interacted with
        non_interacted_blogs = all_blogs[~torch.isin(all_blogs, interacted_blogs)]
        
        # Create user-blog edge index for prediction
        user_id_tensor = torch.tensor([user_id] * len(non_interacted_blogs), dtype=torch.long)
        edge_index = torch.stack((user_id_tensor, non_interacted_blogs.long()))

        # Predict interaction scores for non-interacted blogs
        predictions = model(data.x, edge_index, None).squeeze()
        
        # Get top-k blog recommendations
        top_k_pred = predictions.topk(k=min(top_k, len(non_interacted_blogs)), largest=True)
        recommended_blogs = non_interacted_blogs[top_k_pred.indices]
        
    return recommended_blogs

In [4]:
blog_data_df = pd.read_csv('data/blog_data.csv')
interactions_df = pd.read_csv('data/blog_interactions.csv')

blog_ids_map = {id: i for i, id in enumerate(blog_data_df['blog_id'])}
blog_data_df['blog_id'] = blog_data_df['blog_id'].map(blog_ids_map)
interactions_df['blog_id'] = interactions_df['blog_id'].map(blog_ids_map)

In [5]:
graph_data = torch.load('output/graph_data.pt')
model_state = torch.load('output/gnn_recommender.pth')

In [7]:
model = GNNRecommender(input_dim=graph_data.x.shape[1], hidden_dim=8, output_dim=1)
_ = model.load_state_dict(model_state)

In [25]:
user_id = 3456
user_blogs = get_user_blog_history(graph_data, user_id, blog_data_df, interactions_df)
recommended_blogs = recommend_blogs(model, graph_data, user_id, top_k=5)

In [26]:
print(f'User {user_id} history:')
for blog in user_blogs:
    print(f"[{blog[1]}] {blog[2]}")

User 3456 history:
[web-development] Why PHP Is Still So Important for Web Development?
[web-development] No code = No SEO?
[web-development] Latest Trends — AI, Web & Mobile
[web-development] JavaScript Quick Tutorial
[web-development] Incidental Documentation: Writing docs without writing docs
[web-development] Importance of mocking and stubbing in Tests
[web-development] If Programming Is Hard, Why Is Finding a Job Much Harder
[web-development] Common Patterns in JavaScript
[web-development] Benefits of using Flask over Django for Web Development
[web-development] A Comprehensive Guide to JavaScript Array Operations and Examples
[web-development] 20 java best practices
[backend-development] [ AI ] 讓 ChatGPT 當我的 Software Project 同事?
[backend-development] Why FastAPI is a Future of Python Web Development
[backend-development] Web Stack Weekly — Issue#63
[backend-development] Web Stack Weekly — Issue#57
[backend-development] Top 10 Backend Programming Languages For 2023
[backend-develo

In [27]:
print(f'\nRecommended Blogs for User {user_id}:')
for blog in recommended_blogs.tolist():
    full_blog = blog_data_df.iloc[blog]
    print(f"[{full_blog['topic']}] {full_blog['blog_title']}")


Recommended Blogs for User 3456:
[backend-development] Full Stack Development: An Overview
[backend-development] How to make something like PHP in Python. Part 1
[web-development] The useEffect() hook in React
[web-development] 2 strange React writing methods
[web-development] OSI model (The open systems interconnection model)
